This example formulates and solves the following simple MIP model:
$$
\begin{array}{rl}  
\text{maximize} & x +   y + 2 z\\
\text{s.t.} & x + 2 y + 3 z \leq 4\\
& x +   y       \geq 1\\
&   x, y, z \in \{0,1\}
\end{array}
$$

In [1]:
import numpy as np
import cvxpy as cvx

In [3]:
# define the boolean variable
x = cvx.Variable(3, boolean = True)              #Defines a 0-1 variable. If it was an integer variable, use cvx.Int()

# define the objective vector
c = np.array([1,1,2]);
objective = cvx.Maximize(c*x)

# define the constraints
constraints = [np.array([1,2,3])*x <= 4, np.array([1,1,0])*x >= 1]

In [4]:
prob = cvx.Problem(objective, constraints)
prob.solve(solver=cvx.GUROBI)

Using license file /Users/joehigh/gurobi.lic


3.0

In [5]:
print(x.value)

[1. 0. 1.]


### Portfolio optimization problem with lot constraints

Suppose a portfolio $x = (x_1, \ldots, x_d)^\top$ is feasible if
+ either $x_i = 0$ or $x_i \geq m_i > 0$
+ $\sum_{i=1}^d x_i \leq W$

Reformulation is as follows:
$$ 
\begin{array}{ll}
m_i z_i \leq x_i \leq W z_i, & i = 1, \ldots, d,\\
\sum_{i=1}^d x_i \leq W,\\
z \in \{0,1\}^d
\end{array}
$$

Solve a max-min problem
$$ 
\begin{array}{rll}
\max & \min_{1 \leq k \leq L} c_k^\top x, \\
\mbox{s.t.} & m_i z_i -x_i \leq 0, & i = 1, \ldots, d,\\
& x_i - Wz_i \leq 0, & i = 1, \ldots, d,\\
& \sum_{i=1}^d x_i \leq W,\\
& z \in \{0,1\}^d
\end{array}
$$

In [7]:
import random as rnd # for generating a random objective
rnd.seed(10)

d = 10

L = 6
c = np.zeros((L,d))
for k in range(L):
    c[k,:] = np.array([rnd.random() - rnd.random() for i in range(d)])
m = 0.1
W = 1

x = cvx.Variable(d)
r = cvx.Variable(1)
z = cvx.Variable(d, boolean=True)

objective = cvx.Maximize(r)

constraints = [];
for k in range(L):
    constraints += [c[k,:]*x - r >= 0]

for i in range(d):
    constraints += [m*z[i] - x[i] <= 0, x[i] - W*z[i] <= 0]
constraints += [sum(x) <= W]
prob = cvx.Problem(objective,constraints)

prob.solve(solver=cvx.GUROBI)
if (prob.status == 'optimal'):
    print(prob.value)
    print('x = ' + str(x.value.T))
    print('z = ' + str(z.value.T))

0.1562083042181433
x = [0.42164776 0.         0.         0.28469222 0.18095626 0.11270376
 0.         0.         0.         0.        ]
z = [ 1. -0. -0.  1.  1.  1. -0. -0. -0. -0.]


In [8]:
cvx

<module 'cvxpy' from '//anaconda3/lib/python3.7/site-packages/cvxpy-1.0.24-py3.7-macosx-10.7-x86_64.egg/cvxpy/__init__.py'>